<a href="https://colab.research.google.com/github/GabrielZanotti/Desafio_ImersaoIAlura/blob/main/ImersaoIAlura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando o SDK do Google

In [1]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai

Configurando API

In [15]:
#Configurações iniciais
import os
import google.generativeai as genai
import random
import time

from google.colab import userdata
api_key = userdata.get('secret_key')
genai.configure(api_key=api_key)

Configurações IA

In [3]:
# Configurações de geração e segurança
generation_config = {
  "candidate_count": 1,
  "temperature": 0.7, # Ajuste a temperatura para controlar a criatividade
}

In [4]:
safety_settings = {
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL': 'BLOCK_NONE',
    'DANGEROUS': 'BLOCK_NONE'
}

In [5]:
# Inicializando o modelo gemini-1.0-pro
model = genai.GenerativeModel(model_name='models/gemini-1.0-pro',
                                  generation_config=generation_config,
                                  safety_settings=safety_settings,)

In [6]:
# Iniciando um chat
chat = model.start_chat(history=[])

In [7]:
# Dicionário para armazenar histórico de interações por cidade
historico_interacoes = {}

Código para gerar as respostas como NPC para uso em mesas de RPG

In [18]:
def gerar_resposta_ia(prompt, persona, cidade, nome_aventureiro, chat):
  """Gera uma resposta usando o modelo PaLM 2, considerando o histórico.

  Args:
    prompt: A pergunta ou declaração do usuário.
    persona: A personalidade que a IA deve assumir.
    cidade: A cidade onde a interação ocorre.
    nome_aventureiro: O nome do aventureiro.
    chat: O objeto de chat do modelo.

  Returns:
    A resposta gerada pela IA.
  """
  historico = historico_interacoes.get(cidade, {}).get(nome_aventureiro, "")

  # Verifica se o jogador está perguntando sobre itens
  if any(keyword in prompt.lower() for keyword in ["itens", "armas", "armaduras", "suprimentos"]):
      instrucao_itens = "Lembre-se de incluir as características de jogo entre parênteses quando descrever itens e seus preços.\nExemplo: Espada Curta (1d6 de dano cortante) (10 peças de ouro)\n"
  else:
      instrucao_itens = ""

  prompt_completo = f"""Você é um {persona} em um mundo de fantasia medieval na cidade de {cidade}.
  {instrucao_itens}{historico} {nome_aventureiro}: {prompt}"""

  resposta = chat.send_message(prompt_completo)
  texto_resposta = resposta.text

  time.sleep(8) #Timer pois estava sofrendo com erros da API relacionados a muitas chamdas seguidas

  # Gera anedota aleatória se for o taverneiro
  if persona == "taverneiro medieval" and random.random() < 0.2: # 20% de chance de contar uma anedota
    prompt_anedota = f"Conte uma anedota engraçada que um taverneiro medieval contaria na cidade de {cidade}."
    anedota = chat.send_message(prompt_anedota).text
    texto_resposta += f"\n\nEnquanto isso, ouvi uma história interessante...\n{anedota}"

  historico_interacoes.setdefault(cidade, {})[nome_aventureiro] =  f"{historico} {nome_aventureiro}: {prompt} {persona}: {texto_resposta}"

  return f"\033[32m{texto_resposta}\033[0m" # Texto em verde

Chatbot

In [ ]:
# Loop principal do chatbot
nome_aventureiro = input("Qual o seu nome, aventureiro? ")
cidade = input("Em qual cidade você inicia sua jornada? ")
persona = None  # Variável para armazenar a persona atual

while True:
  print(f"\nVocê está em {cidade}.")

  if persona is None:  # Se não estiver conversando com ninguém
    print("\nQuem você gostaria de encontrar? Escolha entre:")
    print("1. Mercador")
    print("2. Taverneiro")
    print("3. Soldado")
    print("4. Trocar de Cidade")
    print("0. Sair")

    escolha = input("Digite o número da sua escolha: ")

    if escolha in ['1', '2', '3']:
      persona = {
          '1': "mercador medieval",
          '2': "taverneiro medieval",
          '3': "soldado medieval",
      }[escolha]
    elif escolha == '4':
      cidade = input("Digite o nome da nova cidade: ")
    elif escolha == '0':
      print("Adeus, aventureiro!")
      break
    else:
      print("Escolha inválida. Tente novamente.")

  # Interação com o NPC (se persona não for None)
  if persona is not None:
    prompt = input(f"O que você gostaria de dizer ao {persona}? ")
    if prompt.upper() == "SAIR":
      persona = None  # Reseta a persona ao digitar SAIR
    else:
      resposta = gerar_resposta_ia(prompt, persona, cidade, nome_aventureiro, chat)
      print(f"{persona}: {resposta}")